In [ ]:
import pandas as pd
import operator
import time
import numpy as np

In [ ]:
config = {}

config["project_path"] = ".."
config["data_folder_path"] = "data"
config["temp_folder_path"] = "temp"

# input files
config['trade_support_input'] = '202001-04_Trade_Support.xlsx'
config['MDM_mapping'] = 'MDM_mapping.xlsx'
config['brand_to_province_brand_file'] = '202001-05_brand_to_province_brand.xlsx'

# output files
config['spending_trade_support'] = 'spending trade support 202001-04.xlsx'

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + config['trade_support_input']

ts_df = pd.read_excel(input_file_path, dtype=str)

ts_df.columns = ['YearMonth', 'Brand', 'Cost']
ts_df["Cost"] = ts_df["Cost"].astype("float")

In [ ]:
ts_df = ts_df.groupby(by = ['YearMonth', 'Brand'])['Cost'].sum().reset_index()

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["temp_folder_path"] + "/" \
                        + config['brand_to_province_brand_file']

brand_prov_split = pd.read_excel(input_file_path, dtype=str)

brand_prov_split['Split_Ratio'] = brand_prov_split['Split_Ratio'].astype("float")

In [ ]:
ts_prov_df = pd.merge(ts_df, brand_prov_split, on=['YearMonth', 'Brand'], how='left')

### Check for um-mapped records

In [ ]:
ts_prov_df[ts_prov_df['Split_Ratio'] == np.NAN]

In [ ]:
ts_prov_df['Value'] = ts_prov_df['Cost'] * ts_prov_df['Split_Ratio']

ts_prov_df = ts_prov_df[['YearMonth', 'MDM_province_code', 'Brand', 'Value']]

ts_all_df = ts_prov_df.groupby(by = ['YearMonth', 'Brand'])['Value'].sum().reset_index()
ts_all_df['MDM_province_code'] = 'National'

output_df = ts_prov_df.append(ts_all_df, ignore_index=True)

# Write output

In [ ]:
output_df['Cost_type_1'] = 'Trade Support'
output_df['Cost_type_2'] = ''
output_df['Cost_type_3'] = ''
output_df['Cost_type_4'] = ''
output_df['AIP'] = 'P'

output_df = output_df[['YearMonth', 'MDM_province_code', 'Cost_type_1', 'Cost_type_2', 'Cost_type_3', 'Cost_type_4', 'AIP', 'Brand', 'Value']]

output_df.columns = ['Date', 'Province', 'Cost_type_1', 'Cost_type_2', 'Cost_type_3', 'Cost_type_4', 'AIP', 'Brand', 'Spending_value']

In [ ]:
output_file_path = config["project_path"] + "/" \
                        + config["temp_folder_path"] + "/" \
                        + config['spending_trade_support']

output_df.to_excel(output_file_path, index=False)